In [1]:
!nvidia-smi


Tue May 20 15:59:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install sdv


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.mixture import GaussianMixture
from sklearn.feature_selection import SelectFromModel
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from scipy.stats import entropy
from scipy.stats import wasserstein_distance
from scipy.stats import ks_2samp

In [4]:
!wget https://raw.githubusercontent.com/AdenRajput/Comparative_Analysis/main/NSL-KDD/imp_final_df_NSLKDD.csv


--2025-05-20 15:59:49--  https://raw.githubusercontent.com/AdenRajput/Comparative_Analysis/main/NSL-KDD/imp_final_df_NSLKDD.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13344244 (13M) [text/plain]
Saving to: ‘imp_final_df_NSLKDD.csv.2’

imp_final_df_NSLKDD 100%[===================>]  12.73M  --.-KB/s    in 0.09s   

2025-05-20 15:59:50 (142 MB/s) - ‘imp_final_df_NSLKDD.csv.2’ saved [13344244/13344244]



In [5]:
# Load NSL-KDD dataset ( Encoded and Important Features only)
df = pd.read_csv('imp_final_df_NSLKDD.csv')

# Display the first few rows of the dataset to understand its structure
print('originail shape:', df.shape)
df.head()


originail shape: (125973, 26)


,src_bytes,dst_bytes,same_srv_rate,diff_srv_rate,flag_SF,dst_host_srv_count,dst_host_same_srv_rate,logged_in,dst_host_serror_rate,dst_host_diff_srv_rate,...,dst_host_srv_diff_host_rate,level,dst_host_count,dst_host_same_src_port_rate,service_private,srv_diff_host_rate,srv_count,dst_host_srv_rerror_rate,service_domain_u,target
0,491,0,1.00,0.00,True,25,0.17,0,0.00,0.03,...,0.00,20,150,0.17,False,0.00,2,0.00,False,0
1,146,0,0.08,0.15,True,1,0.00,0,0.00,0.60,...,0.00,15,255,0.88,False,0.00,1,0.00,False,0
2,0,0,0.05,0.07,False,26,0.10,0,1.00,0.05,...,0.00,19,255,0.00,True,0.00,6,0.00,False,1
3,232,8153,1.00,0.00,True,255,1.00,1,0.03,0.00,...,0.04,21,30,0.03,False,0.00,5,0.01,False,0
4,199,420,1.00,0.00,True,255,1.00,1,0.00,0.00,...,0.00,21,255,0.00,False,0.09,32,0.00,False,0


In [6]:
# Split the data into features and target
X = df.drop(columns=['target'])
y = df['target']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 1. CTGAN

In [7]:
# CTGAN
# Import necessary packages
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy.stats as stats

# Define metadata for the dataset
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)

# CTGAN
ctgan = CTGANSynthesizer(metadata, epochs=10)
ctgan.fit(df)
data_ctgan = ctgan.sample(len(df))
#X_ctgan = synthetic_data.drop(columns='target').values
#y_ctgan = synthetic_data['target'].values

#X = data.drop(columns=['target'])
#y = data['target']

#df_ctgan = pd.DataFrame(X_ctgan, columns=X.columns)
#df_ctgan['target'] = y_ctgan

data_ctgan.head()

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:145: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:123: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,src_bytes,dst_bytes,same_srv_rate,diff_srv_rate,flag_SF,dst_host_srv_count,dst_host_same_srv_rate,logged_in,dst_host_serror_rate,dst_host_diff_srv_rate,...,dst_host_srv_diff_host_rate,level,dst_host_count,dst_host_same_src_port_rate,service_private,srv_diff_host_rate,srv_count,dst_host_srv_rerror_rate,service_domain_u,target
0,0,6548,0.02,0.06,False,18,0.04,0,1.00,0.08,...,0.0,21,255,0.00,True,0.0,22,0.00,False,1
1,7370,9211,1.00,0.00,True,255,1.00,1,0.00,0.00,...,0.0,21,255,1.00,False,0.0,6,0.01,False,0
2,2051,12832,1.00,0.00,True,255,0.01,1,0.02,0.07,...,0.0,18,255,0.03,False,0.0,4,0.00,False,0
3,12338,3660,1.00,0.00,True,254,1.00,1,0.00,0.00,...,0.0,21,255,0.01,False,0.0,76,0.00,False,0
4,93,12882,0.97,0.00,True,110,0.00,0,0.00,0.02,...,0.0,21,255,0.01,False,0.0,2,0.00,False,0


In [8]:
data_ctgan.to_csv('data_ctgan.csv', index = False)

# 2. GANBLR++

In [9]:
!pip install ganblr

In [ ]:
#from ganblr import get_demo_data
import numpy as np
np.NaN = np.nan  # workaround per codice legacy

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from ganblr.models import GANBLRPP

# to have the same datatype
def is_numerical(dtype):
    return dtype.kind in 'iuf'

# Check for numerical columns in the features only (exclude target column)
column_is_numerical = df.iloc[:, :-1].dtypes.apply(is_numerical).values
numerical_columns = np.argwhere(column_is_numerical).ravel()

# # Ensure numerical_columns are within the bounds of data's feature columns
# if numerical_columns.max() >= df.shape[1] - 1:
#     raise IndexError("Numerical column index out of bounds")

#Initialize the model with the correct numerical columns
model = GANBLRPP(numerical_columns)


# Fit the model
model.fit(df.values[:, :-1], df.values[:, -1], epochs=10)


# Generate synthetic data
synthetic_data = model.sample(1000)
data_ganblrpp = pd.DataFrame(synthetic_data, columns=df.columns)

# Map the data types from the real data to the synthetic data
dtype_mapping = df.dtypes.to_dict()
data_ganblrpp = data_ganblrpp.astype(dtype_mapping)

data_ganblrpp.head()


In [ ]:
data_ganblrpp.to_csv('data_ganblrpp.csv', index = False)

# 3. Copula GAN

In [ ]:
# Define metadata for the dataset
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)

# CopulaGan
from sdv.single_table import CopulaGANSynthesizer

copulagan = CopulaGANSynthesizer(metadata)
copulagan.fit(df)

data_copula_gan = copulagan.sample(len(df))
data_copula_gan.head()

In [ ]:
data_copula_gan.to_csv('data_copula_gan.csv', index = False)

In [ ]:
# 4. CASTGAN

In [ ]:
data_castgan = pd.read_csv("data_castgan.csv")
data_castgan.head()

## Correlation Comparison

In [ ]:
# Calculate correlation matrices
real_corr = df.corr()
ctgan_corr = data_ctgan.corr()
abs_diff_ctgan_corr= (real_corr - ctgan_corr).abs()

garnblr_corr = data_ganblrpp.corr()
abs_diff_garnblr_corr = (real_corr - garnblr_corr).abs()

copula_corr = data_copula_gan.corr()
abs_diff_copula_corr = (real_corr - copula_corr).abs()

castgan_corr = data_castgan.corr()
abs_diff_castgan_corr = (real_corr - castgan_corr).abs()



In [ ]:
#sns.heatmap(corr, cmap="coolwarm", annot=False, xticklabels=corr.columns, yticklabels=corr.columns)
# Plot the correlation matrix of the original and all oversampled data
fig, axes = plt.subplots(2, 3, figsize=(30, 15))
sns.heatmap(df.corr(), ax=axes[0, 0], cmap="coolwarm", annot=False, xticklabels=df.corr().columns, yticklabels=df.corr().columns)
axes[0, 0].set_title("Original Data Correlation Matrix")
sns.heatmap(data_ctgan.corr(), ax=axes[0, 1], cmap="coolwarm", annot=False, xticklabels=data_ctgan.corr().columns, yticklabels=data_ctgan.corr().columns)
axes[0, 1].set_title("CTGAN Correlation Matrix")
sns.heatmap(data_ganblrpp.corr(), ax=axes[0, 2], cmap="coolwarm", annot=False, xticklabels=data_ganblrpp.corr().columns, yticklabels=data_ganblrpp.corr().columns)
axes[0, 2].set_title("GANBLR++ Correlation Matrix")
sns.heatmap(data_copula_gan.corr(), ax=axes[1, 0], cmap="coolwarm", annot=False, xticklabels=data_copula_gan.corr().columns, yticklabels=data_copula_gan.corr().columns)
axes[1, 0].set_title("COPULA GAN Correlation Matrix")
sns.heatmap(data_castgan.corr(), ax=axes[1, 1], cmap="coolwarm", annot=False, xticklabels=data_castgan.corr().columns, yticklabels=data_castgan.corr().columns)
axes[1, 1].set_title("CASTGAN Correlation Matrix")
plt.show()

In [ ]:

# Plot the absolute difference heatmaps
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
sns.heatmap(abs_diff_ctgan_corr, ax=axes[0, 0], cmap="coolwarm", annot=False, xticklabels=abs_diff_ctgan_corr.columns, yticklabels=abs_diff_ctgan_corr.columns)
axes[0, 0].set_title("Absolute Difference with CTGAN")
sns.heatmap(abs_diff_garnblr_corr, ax=axes[0, 1], cmap="coolwarm", annot=False, xticklabels=abs_diff_garnblr_corr.columns, yticklabels=abs_diff_garnblr_corr.columns)
axes[0, 1].set_title("Absolute Difference with GANBLR++")
sns.heatmap(abs_diff_copula_corr, ax=axes[1, 0], cmap="coolwarm", annot=False, xticklabels=abs_diff_copula_corr.columns, yticklabels=abs_diff_copula_corr.columns)
axes[1, 0].set_title("Absolute Difference with COPULA GAN")
sns.heatmap(abs_diff_castgan_corr, ax=axes[1, 1], cmap="coolwarm", annot=False, xticklabels=abs_diff_castgan_corr.columns, yticklabels=abs_diff_castgan_corr.columns)
axes[1, 1].set_title("Absolute Difference with CASTGAN")

plt.show()


## Probability Distribution


In [ ]:
# Define a function to count columns in each DataFrame
def count_columns(dataframes, titles):
    column_counts = {}
    for df, title in zip(dataframes, titles):
        column_counts[title] = len(df.columns)
    return column_counts


# List of DataFrames
dfs = [df, data_ctgan, data_ganblrpp, data_copula_gan, data_castgan]
titles = ["Original Data", "CTGAN", "GANBLR++", "COPULA GAN", "CASTGAN"]

# Count the columns in each DataFrame
column_counts = count_columns(dfs, titles)

# Print the column counts
for title, count in column_counts.items():
    print(f"{title} has {count} columns.")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ks_2samp

# List of DataFrames and their titles
dfs = [df, data_ctgan, data_ganblrpp, data_copula_gan, data_castgan]
titles = ["Original Data", "CTGAN", "GANBLR++", "COPULA GAN", "CASTGAN"]

# Function to compare distributions and calculate KS test results
def compare_distributions(real_data, synthetic_datasets, titles):
    results = {}
    average_results = {}

    for synthetic_data, title in zip(synthetic_datasets, titles):

        ks_results = []

        equal_count = 0
        different_count = 0

        for column in real_data.columns:
            if column == 'target':
                continue

            if column not in synthetic_data.columns:
                print(f"Column {column} not found in dataset {title}. Skipping.")
                continue

            real_values = real_data[column].values
            synthetic_values = synthetic_data[column].values

            # Perform KS Test
            ks_statistic, ks_p_value = ks_2samp(real_values, synthetic_values)

            # Store KS test results
            ks_results.append({
                'Feature': column,
                'KS Statistic': ks_statistic,
                'KS P-value': ks_p_value
            })

            # Count equal and different distributions
            if ks_p_value > 0.05:
                equal_count += 1
            else:
                different_count += 1

        # Store results for current synthetic dataset
        results[title] = ks_results

        # Calculate average results
        total_features = len(real_data.columns) - 1  # excluding target column
        average_results[title] = {
            'Equal Distributions': equal_count,
            'Different Distributions': different_count,
            'Average KS P-value': np.mean([result['KS P-value'] for result in ks_results])
        }

    # Print average results
    print("\n--- Average Results ---")
    for title, avg_res in average_results.items():
        print(f"{title}:")
        print(f"Equal Distributions: {avg_res['Equal Distributions']}")
        print(f"Different Distributions: {avg_res['Different Distributions']}")
        print(f"Average KS P-value: {avg_res['Average KS P-value']}")
        print("-----------------------------")

    return results, average_results

# Function to plot KDE plots for all features across all datasets
def plot_kde_comparison(real_data, synthetic_datasets, titles):
    num_datasets = len(synthetic_datasets)  # excluding original data
    num_features = len(real_data.columns) - 1  # excluding target column

    for column in real_data.columns:
        if column == 'target':
            continue

        plt.figure(figsize=(12, 8))

        # Plot the original data curve
        sns.kdeplot(real_data[column], label='Original Data', linewidth=2, color='black')

        # Plot KDE for each synthetic dataset
        for i, (synthetic_data, title) in enumerate(zip(synthetic_datasets, titles[1:])):  # Skip the original in titles
            if column in synthetic_data.columns:
                sns.kdeplot(synthetic_data[column], label=title)

        plt.title(f'Probability Distribution of {column}')
        plt.legend()
        plt.show()

# Example usage
results, average_results = compare_distributions(df, dfs, titles)
plot_kde_comparison(df, dfs, titles)


In [ ]:

# List of DataFrames
dfs = [df, data_ctgan, data_ganblrpp, data_copula_gan, data_castgan]
titles = ["Original Data", "CTGAN", "GANBLR++", "COPULA GAN", "CASTGAN"]

# Function to compare distributions and calculate KS test results
def compare_distributions(real_data, synthetic_datasets, titles):
    results = {}
    average_results = {}

    for synthetic_data, title in zip(synthetic_datasets, titles):

        ks_results = []

        equal_count = 0
        different_count = 0

        for column in real_data.columns:
            if column == 'target':
                continue

            real_values = real_data[column].values
            synthetic_values = synthetic_data[column].values

            # Perform KS Test
            ks_statistic, ks_p_value = ks_2samp(real_values, synthetic_values)

            # Store KS test results
            ks_results.append({
                'Feature': column,
                'KS Statistic': ks_statistic,
                'KS P-value': ks_p_value
            })

            # Count equal and different distributions
            if ks_p_value > 0.05:
                equal_count += 1
            else:
                different_count += 1

        # Store results for current synthetic dataset
        results[title] = ks_results

        # Calculate average results
        total_features = len(real_data.columns) - 1  # excluding target column
        average_results[title] = {
            'Equal Distributions': equal_count,
            'Different Distributions': different_count,
            'Average KS P-value': np.mean([result['KS P-value'] for result in ks_results])
        }

    # Print average results
    print("\n--- Average Results ---")
    for title, avg_res in average_results.items():
        print(f"{title}:")
        print(f"Equal Distributions: {avg_res['Equal Distributions']}")
        print(f"Different Distributions: {avg_res['Different Distributions']}")
        print(f"Average KS P-value: {avg_res['Average KS P-value']}")
        print("-----------------------------")

    return results, average_results

# Function to plot KDE plots for all features across all datasets
def plot_kde_comparison(real_data, synthetic_datasets, titles, ks_results):
    num_datasets = len(synthetic_datasets) + 1  # including original data
    num_features = len(real_data.columns) - 1  # excluding target column

    for column in real_data.columns:
        if column == 'target':
            continue

        plt.figure(figsize=(12, 8))
        for i, synthetic_data in enumerate(synthetic_datasets):
            sns.kdeplot(synthetic_data[column], label=titles[i])

        #sns.kdeplot(real_data[column], label='Original Data', linewidth=3, color='black')

        # Annotate KS test results on the plot
        #for result in ks_results:
         #   ks_statistic = result['KS Statistic']
          #  ks_p_value = result['KS P-value']
           # if ks_p_value > 0.05:
            #    plt.text(0.98, 0.95 - ks_results.index(result)*0.05, f'{result["Feature"]}: KS p-value={ks_p_value:.3f}', transform=plt.gca().transAxes, ha='right', color='green')
            #else:
             #   plt.text(0.98, 0.95 - ks_results.index(result)*0.05, f'{result["Feature"]}: KS p-value={ks_p_value:.3f}', transform=plt.gca().transAxes, ha='right', color='red')

        plt.title(f'Probability Distribution of {column}')
        plt.legend()
        plt.show()

# Example usage
# Assuming `dfs`, `titles`, and `df` are defined as before
results, average_results = compare_distributions(df, dfs, titles)
plot_kde_comparison(df, dfs, titles, results[titles[1]])  # Display for one synthetic dataset (change index as needed)


## ML Utility

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, f1

# Initialize results dictionary
utility_results = {title: {'TRTR_accuracy': 0, 'TRTR_f1': 0, 'TSTR_accuracy': 0, 'TSTR_f1': 0} for title in titles[1:]}

# Evaluate TRTR (Training on Real, Testing on Real)
model_trtr = RandomForestClassifier(random_state=42)
model_trtr.fit(X_train, y_train)
trtr_accuracy, trtr_f1 = evaluate_model(model_trtr, X_test, y_test)
utility_results["Original Data"] = {'TRTR_accuracy': trtr_accuracy, 'TRTR_f1': trtr_f1}

# Evaluate TSTR (Training on Synthetic, Testing on Real) data
for i, df in enumerate(dfs[1:], 1):
    X_synthetic = df.drop('target', axis=1)
    y_synthetic = df['target']
    model_tstr = RandomForestClassifier(random_state=42)
    model_tstr.fit(X_synthetic, y_synthetic)
    tstr_accuracy, tstr_f1 = evaluate_model(model_tstr, X_test, y_test)
    utility_results[titles[i]]['TSTR_accuracy'] = tstr_accuracy
    utility_results[titles[i]]['TSTR_f1'] = tstr_f1

# Display the utility results
utility_df = pd.DataFrame(utility_results).T
print("Utility results (F1 score, accuracy comparison of TRTR and TSTR) for all datasets:")
print(utility_df)


In [ ]:
for title, df in zip(titles, dfs):
    print(f"DataFrame: {title}")
    print(df.logged_in.value_counts())
    #print(df.protocol_type_icmp.value_counts())
    #print(df.protocol_type_tcp.value_counts())
    print(df.service_http.value_counts())
    print(df.flag_S0.value_counts())
    print(df.service_private.value_counts())
    print(df.target.value_counts())
    print("\n")  # Add a newline for better readability between DataFrames


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# List of DataFrames
dfs = [df, data_ctgan, data_ganblrpp, data_copula_gan, data_castgan]
titles = ["Original Data", "CTGAN", "GANBLR++", "COPULA GAN", "CASTGAN"]

# Ensure that all DataFrames have the same columns
features = df.columns  # List of feature names

# Number of features
num_features = len(features)
num_datasets = len(dfs) - 1  # Number of synthetic datasets

# Create a figure with subplots
fig, axes = plt.subplots(num_features, num_datasets, figsize=(15, 5 * num_features), sharex=True, sharey=True)

# Iterate over each feature
for i, feature in enumerate(features):
    # Data ranges for consistent scaling
    all_data = pd.concat([df[feature]] + [dataset[feature] for dataset in dfs[1:]], axis=0)
    data_min, data_max = all_data.min(), all_data.max()

    for j in range(1, len(dfs)):
        # Plot original data KDE
        sns.kdeplot(df[feature], ax=axes[i, j-1], color='blue', label='Original Data', linestyle='--', bw_adjust=0.5)

        # Plot synthetic dataset KDE
        sns.kdeplot(dfs[j][feature], ax=axes[i, j-1], color='red', label=titles[j], bw_adjust=0.5)

        # Set title and legend
        axes[i, j-1].set_title(f'{feature} - {titles[j]} vs Original')
        axes[i, j-1].legend()

        # Set limits based on all data ranges
        axes[i, j-1].set_xlim(data_min, data_max)
        axes[i, j-1].set_ylim(0, axes[i, j-1].get_ylim()[1] * 1.1)

# Set labels for x and y axes
for ax in axes.flat:
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


dfs = [data_ctgan, data_ganblrpp, data_copula_gan, data_castgan]  # Synthetic DataFrames
titles = ["CTGAN", "GANBLR++", "COPULA GAN", "CASTGAN"]  # Titles for the synthetic datasets

# Ensure that all DataFrames have the same columns
features = df.columns  # List of feature names

# Create a figure with subplots
fig, axes = plt.subplots(len(features), len(dfs), figsize=(20, 5 * len(features)), sharex=False, sharey=False)

for i, feature in enumerate(features):
    # Data ranges for individual scaling
    all_data = pd.concat([df[feature]] + [dataset[feature] for dataset in dfs], axis=0)
    data_min, data_max = all_data.min(), all_data.max()

    # Plot KDE for each synthetic dataset
    for j, (synthetic_data, title) in enumerate(zip(dfs, titles)):
        if feature in synthetic_data.columns:
            # Plot original data KDE
            sns.kdeplot(df[feature], ax=axes[i, j], color='blue', label='Original Data', linestyle='--', bw_adjust=0.5)

            # Plot synthetic dataset KDE
            sns.kdeplot(synthetic_data[feature], ax=axes[i, j], color='red', label=title, bw_adjust=0.5)

            # Set title and legend
            axes[i, j].set_title(f'{feature} - {title} vs Original')
            axes[i, j].legend()

            # Set limits based on individual feature data
            axes[i, j].set_xlim(data_min, data_max)
            axes[i, j].set_ylim(0, axes[i, j].get_ylim()[1] * 1.1)
        else:
            # Handle missing feature in synthetic datasets
            axes[i, j].text(0.5, 0.5, 'No Data', ha='center', va='center', fontsize=12)
            axes[i, j].set_title(f'{feature} - {title} vs Original')
            axes[i, j].set_xlim(data_min, data_max)
            axes[i, j].set_ylim(0, 1)

# Set labels for x and y axes
for ax in axes.flat:
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')

# Adjust layout
plt.tight_layout()
plt.show()
